# Import packages

In [4]:
!pip install -r multiff_analysis/requirements.txt

  Cloning https://github.com/gallantlab/pyrcca.git to /private/var/folders/81/249hbjx960s5t_1dxgdxz6r00000gn/T/pip-req-build-4axpc730
  Running command git clone --filter=blob:none --quiet https://github.com/gallantlab/pyrcca.git /private/var/folders/81/249hbjx960s5t_1dxgdxz6r00000gn/T/pip-req-build-4axpc730
  Resolved https://github.com/gallantlab/pyrcca.git to commit bc60f25a8e395baf495dd0f7234a68490ed16a22
  Preparing metadata (setup.py) ... done


In [6]:
!pip install pygam

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pygam]


In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

project_folder = '/Users/dusiyi/Documents/Multifirefly-Project'
os.chdir(project_folder)
sys.path.append(os.path.join(project_folder, 'multiff_analysis', 'methods'))

from data_wrangling import specific_utils, process_monkey_information
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from non_behavioral_analysis.neural_data_analysis.get_neural_data import neural_data_processing
from non_behavioral_analysis.neural_data_analysis.visualize_neural_data import plot_neural_data, plot_modeling_result
from non_behavioral_analysis.neural_data_analysis.model_neural_data import cca_class, pgam_class, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from non_behavioral_analysis.neural_data_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from non_behavioral_analysis.neural_data_analysis.planning_neural import planning_neural_class, planning_neural_utils
from non_behavioral_analysis.neural_data_analysis.decode_targets import behav_features_to_keep, decode_target_class, plot_gpfa_utils, decode_target_utils, fit_gpfa_utils, gpfa_regression_utils

import sys
import math
import gc
import subprocess
from pathlib import Path
from importlib import reload

# Third-party imports
import numpy as np
from numpy import var
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca

# To fit gpfa
import numpy as np
from scipy.integrate import odeint
import quantities as pq
import neo
from elephant.spike_train_generation import inhomogeneous_poisson_process
from elephant.gpfa import GPFA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from elephant.gpfa import gpfa_core, gpfa_util

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")


pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)


%load_ext autoreload
%autoreload 2

RuntimeError: The new DType API is currently in an exploratory phase and should NOT be used for production code.  Expect modifications and crashes!  To experiment with the new API you must set `NUMPY_EXPERIMENTAL_DTYPE_API=1` as an environment variable.

RuntimeError: cannot load _umath_tests module.

# Get data

In [2]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0416"
dec = decode_target_class.DecodeTargetClass(raw_data_folder_path=raw_data_folder_path,
                                                               bin_width=0.02, window_width=0.05)

Retrieved monkey_information
The number of points that were removed due to delta_position exceeding the ceiling is 0
Note: ff_caught_T_sorted is replaced with ff_caught_T_new
Retrieved ff_dataframe from all_monkey_data/processed_data/monkey_Schro/data_0416/ff_dataframe.h5


In [3]:
dec.behav_data.shape

AttributeError: 'DecodeTargetClass' object has no attribute 'behav_data'

In [ ]:
stop!

In [4]:
dec.streamline_making_behav_and_neural_data()
dec.get_x_and_y_var()
dec.pursuit_data.head(3)

Retrieved target_df
Warnings: At least one ff has a lower bound equal to the upper bound. Please check the input.
monkey_y_target_last_seen_frozen             10651
monkey_angle_target_last_seen_frozen         10651
cum_distance_target_last_seen_frozen         10651
distance_from_monkey_pos_target_last_seen    10651
cum_distance_since_target_last_seen          10651
d_heading_since_target_last_seen             10651
dtype: int64
Retrieved target_clust_last_vis_df
Percentage of targets not in a visible cluster out of all targets 61.48936170212767
10812 rows of 183061 rows (5.9%) of behav_data_all are preserved after taking out chunks between target last-seen time and capture time
0 segments (0.0%) out of 289 segments have 0 duration. They are dropped from pursuit data
Window width changed from 0.05 to 0.06 to make it odd


,bin,point_index,monkey_speed,monkey_angle,monkey_dw,...,gaze_world_x_r,gaze_world_y_r,target_index,target_x,target_y
0,1161,1393.00000,200.00000,2.25388,-0.00006,...,351.83241,364.51694,1,825.24321,302.26639
1,1162,1394.00000,86.35241,2.25535,0.08863,...,427.21394,321.36366,1,825.24321,302.26639
2,1163,1395.00000,200.00000,2.25593,0.03467,...,510.20093,278.03411,1,825.24321,302.26639


In [4]:
# Show rows with NA values in pursuit_data_all
na_rows = dec.pursuit_data_all[dec.pursuit_data_all.isna().any(axis=1)]
print("Number of rows with NA values:", len(na_rows))
print("\nColumns with NA values:")
print(dec.pursuit_data_all.isna().sum()[dec.pursuit_data_all.isna().sum() > 0])
print("\nSample of rows with NA values:")
print(na_rows.head())


Number of rows with NA values: 1

Columns with NA values:
monkey_x_target_last_seen_frozen             1
monkey_y_target_last_seen_frozen             1
monkey_angle_target_last_seen_frozen         1
cum_distance_target_last_seen_frozen         1
distance_from_monkey_pos_target_last_seen    1
cum_distance_since_target_last_seen          1
d_heading_since_target_last_seen             1
dtype: int64

Sample of rows with NA values:
          bin   monkey_x   monkey_y       time  point_index  monkey_angle  \
10347  118192 -690.05548 -687.94037 2363.84340       142767       1.70764   

       monkey_speed  monkey_dw      RDz  monkey_y_smr  ...  \
10347       0.00000    0.24516 -4.83786    -687.89526  ...   

       target_last_seen_time_abs  distance_from_monkey_pos_target_last_seen  \
10347                 2463.84340                                        NaN   

       cum_distance_since_target_last_seen  d_heading_since_target_last_seen  \
10347                                  NaN       

### debug

In [ ]:
dec.y_var_lags['bin_0']

In [ ]:
dec.get_x_and_y_var()

In [ ]:
dec.x_var_lags.shape

(179947, 946)

In [ ]:
dec.behav_data_all.shape

(183061, 87)

In [ ]:
dec.behav_data_all.bin.unique().shape

(179947,)

In [ ]:
import gc
gc.collect()

3

In [ ]:
import subprocess
result = subprocess.run(['ps', 'aux'], capture_output=True, text=True)
python_processes = [line for line in result.stdout.split('\n') if 'python' in line.lower()]
for proc in python_processes:
    print(proc)

dusiyi           89281   0.9  2.5 1627003552 419888   ??  S    10:21AM   1:24.35 /Applications/Cursor.app/Contents/Frameworks/Cursor Helper (Plugin).app/Contents/MacOS/Cursor Helper (Plugin) /Users/dusiyi/.cursor/extensions/ms-python.vscode-pylance-2024.8.1/dist/server.bundle.js --cancellationReceive=file:84d8ac090c9f34809079fa87a1496ce4ac8c2d2962 --node-ipc --clientProcessId=89098
dusiyi           89215   0.0  0.1 411180272  15776   ??  S    10:20AM   0:01.35 /opt/homebrew/Cellar/python@3.10/3.10.16/Frameworks/Python.framework/Versions/3.10/Resources/Python.app/Contents/MacOS/Python /Users/dusiyi/.cursor/extensions/ms-python.isort-2025.0.0/bundled/tool/lsp_server.py
dusiyi           89214   0.0  0.1 411197840  13920   ??  S    10:20AM   0:01.30 /opt/homebrew/Cellar/python@3.10/3.10.16/Frameworks/Python.framework/Versions/3.10/Resources/Python.app/Contents/MacOS/Python /Users/dusiyi/.cursor/extensions/ms-python.black-formatter-2025.2.0/bundled/tool/lsp_server.py --stdio
dusiyi         

In [ ]:
!kill -9 97368

: 

In [ ]:
# In Jupyter
%who
%whos
%reset -f  # Clear all variables if needed

Axes3D	 CCA	 CanCorr	 GPFA	 GridSearchCV	 LinearRegression	 Path	 Ridge	 StandardScaler	 
behav_features_to_keep	 builtins	 cca_class	 cluster_analysis	 debugpy	 dec	 decode_target_class	 decode_target_utils	 drop_high_corr_vars	 
drop_high_vif_vars	 fftconvolve	 fit_gpfa_utils	 gc	 gpfa_core	 gpfa_regression_utils	 gpfa_util	 inhomogeneous_poisson_process	 interpolate	 
ipykernel	 linalg	 loadmat	 math	 matplotlib	 mean_squared_error	 neo	 neural_data_modeling	 neural_data_processing	 
neural_vs_behavioral_class	 np	 odeint	 organize_patterns_and_features	 os	 pattern_by_trials	 pd	 pgam_class	 pi	 
planning_neural_class	 planning_neural_utils	 plot_behaviors_utils	 plot_gpfa_utils	 plot_modeling_result	 plot_neural_data	 plt	 pq	 prep_monkey_data	 
prep_target_data	 proc	 process_monkey_information	 project_folder	 python_processes	 r2_score	 raw_data_folder_path	 rc	 rcca	 
reload	 result	 sns	 sparse	 specific_utils	 subprocess	 sys	 torch	 train_test_split	 
var	 variance_inflatio

In [ ]:
# In Jupyter
%who
%whos
%reset -f  # Clear all variables if needed

In [ ]:
type(dec.behav_data_all)

NameError: name 'df' is not defined

In [ ]:
dec.y_var.bin.shape

(10812,)

In [ ]:
dec.y_var.bin.unique().shape

(10752,)

In [5]:
import gc
gc.collect()

0

In [6]:
stop!

SyntaxError: invalid syntax (3319058519.py, line 1)

# GPFA

## get data for GPFA

In [ ]:
dec.single_vis_target_df.columns

In [ ]:
dec.prepare_spikes_for_gpfa()

print(len(dec.spiketrains))
print(len(dec.spiketrains[0]))
print(len(dec.spiketrains[0][2]))

## fit gpfa

In [ ]:
len(dec.spiketrains)

In [ ]:
dec.get_gpfa_trahjectories(latent_dimensionality=10)


In [ ]:
stop!

## plot latent dimensions

In [ ]:
plot_gpfa_utils.plot_gpfa_traj_3d_uniform_color()


In [ ]:
# First, enable interactive mode in your notebook
%matplotlib inline

# Import required modules
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

# Create the interactive plot
fig, ax = plot_gpfa_utils.plot_gpfa_traj_3d(
    trajectories=dec.trajectories,
    figsize=(15, 5),
    linewidth_single_trial=0.75,
    alpha_single_trial=0.3,
    linewidth_trial_average=2,
    title='Latent dynamics extracted by GPFA',
    view_azim=-5,
    view_elev=60
)

plt.show()

In [ ]:
# fig = plot_gpfa_utils.plot_gpfa_traj_3d_plotly(trajectories)

In [ ]:
# Find variance explained by each latent dimension
traj_stack = np.stack(dec.trajectories, axis=0)  # shape: (n_trials, 3, T)
var_by_dim = var(traj_stack, axis=(0, 2))    # variance across trials and time
var_by_dim /= var_by_dim.sum()               # normalize to get explained variance ratio
print("Variance explained by each latent dimension:", var_by_dim)


In [ ]:

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(15, 5))

ax.set_title('Latent dynamics extracted by GPFA')
ax.set_xlabel('Time [s]')

average_trajectory = np.mean(, axis=0)
time = np.arange(len(average_trajectory[0])) * 0.02  # assuming all trajectories have the same length

for i, x in enumerate(average_trajectory):
    ax.plot(time, x, label=f'Dim {i+1}')

ax.legend()

plt.tight_layout()
plt.show()


## prepare data for regression

## regression

In [ ]:
dec.common_t_stop = max(dec.spike_segs_df['t_duration']) * pq.s + dec.bin_width_w_unit        

In [ ]:
dec.get_latent_data_and_behav_data_for_all_trials()

In [ ]:
max_timepoints = int(1.5/0.02)
scores_by_time, times = gpfa_regression_utils.time_resolved_regression_variable_length(dec.latent_trials, dec.behavior_trials, time_step=0.02, cv_folds=5, max_timepoints=max_timepoints)

In [ ]:
behavior_labels = dec.pursuit_data_by_trial.drop(columns=['segment']).columns
len(behavior_labels)
scores_by_time_df = pd.DataFrame(scores_by_time, columns=behavior_labels)
# see the percentage of 1 of this dummy variable
dec.pursuit_data_by_trial[['whether_new_distinct_stop']].sum()/len(dec.pursuit_data_by_trial)

In [ ]:
gpfa_regression_utils.plot_time_resolved_scores(scores_by_time, times, behavior_labels=behavior_labels)


In [ ]:
trial_counts = np.array([sum(latent.shape[0] > t for latent in latent_trials)
                         for t in range(scores_by_time.shape[0])])
plt.plot(times, trial_counts)
plt.xlabel("Time (s)")
plt.ylabel("Trials with data")


# Reduce columns in lags

In [ ]:
dec.reduce_x_var_lags()
# dec.reduce_y_var_lags() # currently not needed bc of the low correlations between clusters or lags

## check result of reducing

In [ ]:
## also check correlations between x vars without lags
high_corr_pair_df, top_n_corr_df = drop_high_corr_vars.get_pairs_of_columns_w_high_corr(
            dec.x_var, corr_threshold=0.8)
top_n_corr_df

# Linear regression (didn't modify yet)

Regressing the behavioral variables individually (as y_var) against all neural activity

## put results in df

In [ ]:
# dec.make_or_retrieve_y_var_lr_resault_df(exists_ok=True)
dec.y_var_lr_result_df = neural_data_modeling.get_y_var_lr_result_df(
                dec.x_var_lags_reduced, dec.y_var)
dec.y_var_lr_result_df.head(5)

##  plot all neural clusters vs one behavioral var

In [ ]:
# drop existing columns involving bin (most likely there's only one or zero after being reduced, because different lags of bins can have very high correlations)
bin_cols = [col for col in dec.y_var_lags_reduced.columns if 'bin' in col]
dec.y_var_lags_reduced.drop(columns=bin_cols, inplace=True)

# then we add the variable bin (so that only the 0 lag is used)
dec.y_var_lags_reduced['bin'] = dec.y_var_lags['bin_0'].astype(int)

In [10]:
dec.y_var.columns

Index(['bin', 'monkey_speed', 'monkey_angle', 'monkey_dw', 'monkey_ddw',
       'monkey_ddv', 'monkey_speeddummy', 'whether_new_distinct_stop',
       'delta_distance', 'LDy', 'LDz', 'RDy', 'RDz', 'gaze_mky_view_x_l',
       'gaze_mky_view_y_l', 'gaze_mky_view_angle_l', 'gaze_mky_view_x_r',
       'gaze_mky_view_y_r', 'gaze_mky_view_angle_r', 'eye_world_speed',
       'valid_view_point_l', 'valid_view_point_r', 'num_alive_ff',
       'num_visible_ff', 'min_ff_distance', 'min_abs_ff_angle',
       'min_abs_ff_angle_boundary', 'min_visible_ff_distance',
       'min_abs_visible_ff_angle', 'min_abs_visible_ff_angle_boundary',
       'catching_ff', 'any_ff_visible', 'target_distance', 'target_angle',
       'target_angle_to_boundary', 'target_rel_x', 'target_rel_y',
       'time_since_target_last_seen', 'target_last_seen_distance_frozen',
       'target_last_seen_angle_frozen',
       'target_last_seen_angle_to_boundary_frozen', 'target_visible_dummy',
       'time_since_last_capture', 'tra

In [9]:
dec.y_var_lags.head(3)

,bin_0,monkey_speed_0,monkey_angle_0,monkey_dw_0,monkey_ddw_0,monkey_ddv_0,monkey_speeddummy_0,whether_new_distinct_stop_0,delta_distance_0,LDy_0,...,cum_distance_since_target_last_seen_5,d_heading_since_target_last_seen_5,monkey_x_5,monkey_y_5,gaze_world_x_l_5,gaze_world_y_l_5,gaze_world_x_r_5,gaze_world_y_r_5,target_x_5,target_y_5
0,6.00000,0.00000,1.57080,188.45962,0.00000,0.00000,0.00000,1.00000,0.00000,23.78424,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,7.00000,0.00000,-1.58170,188.45962,-94.22981,0.00000,0.00000,0.00000,0.00000,23.76074,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,8.00000,0.00000,-1.58170,0.00000,-94.22981,0.00000,0.00000,0.00000,0.00000,23.74792,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [12]:
dec.binned_spikes_df.head()

,unit_0,unit_1,unit_2,unit_3,unit_4,unit_5,unit_6,unit_7,unit_8,unit_9,...,unit_77,unit_78,unit_79,unit_80,unit_81,unit_82,unit_83,unit_84,unit_85,bin
0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0
1,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1
2,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,2
3,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,3
4,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,4


In [13]:
dec.binned_spikes_df.shape

(179948, 87)

In [18]:
dec.binned_spikes_df[['bin']].tail()

,bin
179943,179943
179944,179944
179945,179945
179946,179946
179947,179947


In [17]:
dec.behav_data.tail(4)

,bin,point_index,monkey_speed,monkey_angle,monkey_dw,monkey_ddw,monkey_ddv,monkey_speeddummy,whether_new_distinct_stop,delta_distance,...,monkey_x,monkey_y,cum_distance,gaze_world_x_l,gaze_world_y_l,gaze_world_x_r,gaze_world_y_r,target_index,target_x,target_y
179510,179944,217186,0.02212,-2.26194,0.00000,0.00000,-4.71356,0,True,0.00037,...,-979.14606,-129.46983,447631.74684,-974.14341,-87.26228,-986.79915,-88.91813,469,-979.96664,-139.99172
179511,179945,217187,0.00000,-2.26194,0.00000,0.00000,-0.01106,0,False,0.00000,...,-979.14606,-129.46983,447631.74684,-1018.01207,-156.05205,-986.79915,-88.91813,469,-979.96664,-139.99172
179512,179946,217189,0.00000,-2.26194,0.00000,0.00000,0.00000,0,False,0.00000,...,-979.14606,-129.46983,447631.74684,-1019.81257,-157.18687,-986.79915,-88.91813,470,-345.51603,824.13004
179513,179947,217190,0.00000,-2.26194,0.00000,0.00000,0.00000,0,False,0.00000,...,-979.14606,-129.46983,447631.74684,-1020.34289,-157.50349,-986.79915,-88.91813,470,-345.51603,824.13004


In [19]:
dec.behav_data_all.shape

(179514, 87)

In [11]:
dec.binned_spikes_df.columns

Index(['unit_0', 'unit_1', 'unit_2', 'unit_3', 'unit_4', 'unit_5', 'unit_6',
       'unit_7', 'unit_8', 'unit_9', 'unit_10', 'unit_11', 'unit_12',
       'unit_13', 'unit_14', 'unit_15', 'unit_16', 'unit_17', 'unit_18',
       'unit_19', 'unit_20', 'unit_21', 'unit_22', 'unit_23', 'unit_24',
       'unit_25', 'unit_26', 'unit_27', 'unit_28', 'unit_29', 'unit_30',
       'unit_31', 'unit_32', 'unit_33', 'unit_34', 'unit_35', 'unit_36',
       'unit_37', 'unit_38', 'unit_39', 'unit_40', 'unit_41', 'unit_42',
       'unit_43', 'unit_44', 'unit_45', 'unit_46', 'unit_47', 'unit_48',
       'unit_49', 'unit_50', 'unit_51', 'unit_52', 'unit_53', 'unit_54',
       'unit_55', 'unit_56', 'unit_57', 'unit_58', 'unit_59', 'unit_60',
       'unit_61', 'unit_62', 'unit_63', 'unit_64', 'unit_65', 'unit_66',
       'unit_67', 'unit_68', 'unit_69', 'unit_70', 'unit_71', 'unit_72',
       'unit_73', 'unit_74', 'unit_75', 'unit_76', 'unit_77', 'unit_78',
       'unit_79', 'unit_80', 'unit_81', 'unit_82',

In [8]:
dec.x_var_lags.head(3)

,unit_0_0,unit_1_0,unit_2_0,unit_3_0,unit_4_0,unit_5_0,unit_6_0,unit_7_0,unit_8_0,unit_9_0,...,unit_76_5,unit_77_5,unit_78_5,unit_79_5,unit_80_5,unit_81_5,unit_82_5,unit_83_5,unit_84_5,unit_85_5
0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [ ]:
dec.binned_spikes_df.shape

In [ ]:
dec.all_binned_spikes.head()

In [ ]:
dec.all_binned_spikes

In [ ]:
dec.x_var.shape

(10746, 86)

In [ ]:
dec.y_var.shape

In [ ]:
dec.y_var_lags.shape

In [ ]:
dec.x_var_lags_reduced.shape

In [ ]:
dec.y_var_lags_reduced.shape

In [ ]:


# conduct linear regression on X and y
plt.rcParams["figure.figsize"] = (20, 10)
bins_to_plot = dec.y_var_lags_reduced['bin'].values
for i, column in enumerate(dec.y_var_lags_reduced.columns):

    plot_neural_data.plot_regression(dec.y_var_lags_reduced, column, dec.x_var_lags_reduced, bins_to_plot=bins_to_plot, min_r_squared_to_plot=0.3)
    # if i == 3:
    #     break

##  plot one neural cluster vs one behavioral var

In [ ]:
# plot one neural cluster against one behavioral variable
cluster_num, behavioral_column = 6, 'monkey_speed'
bins_to_plot = range(1000, 1200)
x_values = dec.binned_spikes_df.loc[bins_to_plot, f'unit_{cluster_num}'].values
y_values = dec.pursuit_data[behavioral_column][bins_to_plot]
reg = LinearRegression().fit(x_values.reshape(-1, 1), y_values)

plt.scatter(x_values, y_values, color='blue', s=1)
plt.plot(x_values, reg.predict(x_values.reshape(-1, 1)), color='red', linewidth=1)
plt.show()

# CCA

https://medium.com/@pozdrawiamzuzanna/canonical-correlation-analysis-simple-explanation-and-python-example-a5b8e97648d2

## No lagging

In [ ]:
cca_no_lag = cca_class.CCAclass(X1=dec.x_var, X2=dec.y_var_reduced, lagging_included=False)

In [ ]:
cca_no_lag.conduct_cca()

## with lags

In [ ]:
cca_lags = cca_class.CCAclass(X1=dec.x_var_lags, X2=dec.y_var_lags_reduced, lagging_included=True)

In [ ]:
dec.y_var_lags_reduced.shape

In [ ]:
cca_lags.conduct_cca()

In [ ]:
gc.collect()

## compare lag vs no lag

In [ ]:
canon_df = pd.DataFrame(cca_no_lag.canon_corr, columns = ['no_lag'])
canon_df[f'lag_{dec.max_lag_number}'] = cca_lags.canon_corr
canon_df['component'] = [f'CC {i+1}' for i in range(cca_lags.n_components)]
# convert canon_df to long format
canon_df_long = pd.melt(canon_df, id_vars=['component'], var_name='lag', value_name='canon_coeff')

In [ ]:
# make a sns bar plot on canon_df_long
plt.figure(figsize=(8, 6))
sns.barplot(x='component', y='canon_coeff', data=canon_df_long, hue='lag')
plt.show()

## cca_inst (choose one between lags and no lag)

In [ ]:
# choose lags
cca_inst = cca_lags

In [ ]:
# choose no lag
cca_inst = cca_no_lag

## loadings

### neurons

In [ ]:
cca_inst.plot_ranked_loadings(X1_or_X2='X1', squared=False)

### behavior

In [ ]:
cca_inst.plot_ranked_loadings(X1_or_X2='X2', squared=False)

## squared loadings

### neurons

In [ ]:
cca_inst.plot_ranked_loadings(X1_or_X2='X1')

### behavior

In [ ]:
cca_inst.plot_ranked_loadings(X1_or_X2='X2')

## abs weights ranked

### neurons

In [ ]:
cca_inst.plot_ranked_weights()

### behavior

In [ ]:
cca_inst.plot_ranked_weights(X1_or_X2='X2')

## plot real weights

### neurons

In [ ]:
cca_inst.plot_ranked_weights(abs_value=False)

### behavior

In [ ]:
cca_inst.plot_ranked_weights(X1_or_X2='X2', abs_value=False)

In [ ]:
stop here!

## distribution of each feature

In [ ]:
cca_inst.X2_sc.shape

In [ ]:
X2_sc_df = pd.DataFrame(cca_inst.X2_sc, columns = cca_inst.X2.columns)
X2_sc_df.describe()

In [ ]:
for column in X2_sc_df.columns:
    plt.figure(figsize=(8, 2))
    sns.boxplot(X2_sc_df[column], orient='h')
    plt.show()
    

## heatmap of weights
raw canonical coefficients are interpreted in a manner analogous to interpreting regression coefficients. For example: a one unit increase in reading leads to a .0446 decrease in the first canonical variate of set 2 when all of the other variables are held constant (in some other data)

In [ ]:
weight_df = cca_inst.X2_weight_df.copy()
weight_df = weight_df.set_index('feature').drop(columns='feature_category')

In [ ]:
plt.subplots(figsize=(15, 25))
sns.heatmap(weight_df.iloc[:20, :10], cmap='coolwarm', annot=True, linewidths=1)
plt.show()

## train test split

In [ ]:
train1, test1, train2, test2 = train_test_split(cca_inst.X1_sc, cca_inst.X2_sc, test_size=0.3, random_state=42)
# use training and testing set
nComponents = 10
cca2 = rcca.CCA(kernelcca = False, reg = 0., numCC = nComponents)
cca2.train([train1, train2])
testcorrs = cca2.validate([test1, test2])
testcorrs

## compute explained variance

In [ ]:
cca2.compute_ev([test1, test2])

## test for p values

In [ ]:
stats_cca = CanCorr(cca_inst.X1_sc, cca_inst.X2_sc)
print(stats_cca.corr_test().summary())
neural_data_modeling.print_weights('X', stats_cca.x_cancoef)
neural_data_modeling.print_weights('Z', stats_cca.y_cancoef)

In [ ]:
stop pls

# PGAM (unfinished)

In [ ]:
## Categorize variables
dec.y_var_reduced.columns
temporal_vars = ['time_rel_to_stop',
 'time_when_nxt_ff_first_seen_rel_to_stop',
 'time_when_cur_ff_first_seen_rel_to_stop',
 'time_when_nxt_ff_last_seen_rel_to_stop',
 'time_when_cur_ff_last_seen_rel_to_stop',
 ]

spatial_vars = [x for x in dec.y_var_reduced.columns if x not in temporal_vars]
spatial_vars

# Inspect data

## sparsity of neural data

In [ ]:
dec.binned_spikes_df.shape

In [ ]:
# inspect neural data

bins = dec.binned_spikes_df

# Calculate percentage of non-zero rows for each column
non_zero_percentages = (bins != 0).mean() * 100

# Create a DataFrame with the results
non_zero_df = pd.DataFrame({
    'Column': non_zero_percentages.index,
    'Percent_Non_Zero': non_zero_percentages.values
})

# Sort by percentage in descending order
non_zero_df = non_zero_df.sort_values('Percent_Non_Zero', ascending=False)

print("Percentage of non-zero values in each column:")
print(non_zero_df)


In [ ]:
bins.drop(columns='bin').mean(axis=1).describe()

# plot the percentile of values of mean firing rates across neurons at each time bin
mean_rates = bins.drop(columns='bin').mean(axis=1)

# Calculate percentiles from 0 to 100
percentiles = np.arange(0, 101, 1)
percentile_values = np.percentile(mean_rates, percentiles)

# Create plot
plt.figure(figsize=(6, 4))
plt.plot(percentiles, percentile_values)
plt.xlabel('Percentile')
plt.ylabel('Mean Firing Rate')
plt.title('Distribution of Mean Firing Rates Across Neurons')
plt.grid(True)
plt.show()


## multicollinearity

### y var (behavioral)

In [ ]:
y_var_vif = drop_high_vif_vars.get_vif_df(dec.y_var)
print(y_var_vif.head(8))

# calculate the correlation coefficient among the columns with VIF > 5
# specific_columns = vif_df[vif_df['vif'] > 5].feature.values
specific_columns = y_var_vif.feature.values[:10]
corr_coeff = dec.y_var[specific_columns].corr()
#plt.figure(figsize = (6, 6))
plt.figure(figsize = (8, 6))
sns.heatmap(corr_coeff, cmap='coolwarm', annot=True, linewidths=1, vmin=-1)
plt.show()

In [ ]:
# Try y_var_reduced

y_var_vif = drop_high_vif_vars.get_vif_df(dec.y_var_reduced)
print(y_var_vif.head(8))

# calculate the correlation coefficient among the columns with VIF > 5
# specific_columns = vif_df[vif_df['vif'] > 5].feature.values
specific_columns = y_var_vif.feature.values[:10]
corr_coeff = dec.y_var[specific_columns].corr()
#plt.figure(figsize = (6, 6))
plt.figure(figsize = (8, 6))
sns.heatmap(corr_coeff, cmap='coolwarm', annot=True, linewidths=1, vmin=-1)
plt.show()

## plot trial segments in pursuit_data

In [ ]:
from visualization.matplotlib_tools import plot_trials,
dec.make_PlotTrials_args()

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]                     

max_plot_to_make = 2
plot_counter = 0

for index, row in dec.single_vis_target_df.iloc[2:].iterrows():

    duration = [row['last_vis_time'], row['ff_caught_time']]

    returned_info = plot_trials.PlotTrials(
                duration, 
                *dec.PlotTrials_args,  
                adjust_xy_limits=True,       
                minimal_margin=50,
                show_reward_boundary=True,
                show_alive_fireflies=False,
                show_visible_fireflies=True,
                show_in_memory_fireflies=True,
                show_believed_target_positions=True,
                )
    plt.show()
    

    plot_counter += 1
    if plot_counter >= max_plot_to_make:
        break

### check target_rel_x and y
(The look correct after checking)

In [ ]:
pursuit_sub = dec.pursuit_data.loc[dec.pursuit_data['target_index']==65].copy()
pursuit_sub['target_angle_deg'] = pursuit_sub['target_angle'] * 180/pi 

In [ ]:
pursuit_sub[['point_index', 'target_angle_deg', 'target_distance', 'target_rel_x', 'target_rel_y']]

# Appendix

## more columns (possibly get in the future)

get also get: (but to be honest, it doesn't make that much sense to get them....so let's skip for now.)
'distance traversed since target last visible',
'd angle since target last visible', 'target_at_right',
'time_till_capture', 'time from last visible to capture

Note that there might be multicollinearity. For example, duration from last visible to capture = time since target last visible + time till capture

Similarly, target angle = target angle last seen frozen - d angle since target last visible

(For distance it's not exactly the same because of the difference between distance and distance traversed, but it's still similar)

The multicollinearity is fine in linear regression (when each feature here is a y var), but need to be dealt with in cca.

## other thoughts

should i actually align each section, as if they are trials???
maybe i can try both that and continuous time... both can shed light on different behavioral variables
but for aligning trials, it may require alignment or warping since trial durations vary.

btw, what does it mean stitch data?

also, what does it look like to use RNN to model it?
I thought about the paper that Noah presented on


btw.......IME

## why ratio of bin/target_index approaches constant

In [ ]:
trial_lengths = dec.pursuit_data[['target_index', 'bin']].groupby('target_index').count()
trial_lengths.describe()

In [ ]:
sub = dec.y_var_reduced[['time', 'bin', 'target_index']]
sub['factor'] = dec.y_var_reduced['bin']/dec.y_var_reduced['target_index']
sub

In [ ]:
plt.hist(np.diff(dec.ff_caught_T_sorted), bins=30)
plt.xlabel('Time difference')
plt.ylabel('Count')
plt.title('Distribution of time differences between caught events')
plt.show()


In [ ]:
dec.ff_caught_T_sorted/np.arange(len(dec.ff_caught_T_sorted))

## compared with neural_data_modeling

In [ ]:
dec = neural_vs_behavioral_class.NeuralVsBehavioralClass(raw_data_folder_path=raw_data_folder_path)
dec.streamline_preparing_neural_and_behavioral_data()

In [ ]:
dec.final_behavioral_data

In [ ]:
dec.y_var

In [ ]:
dec.y_var_reduced

In [ ]:
dec.y_var.columns

In [ ]:
dec.y_var_reduced.columns

In [ ]:
[col for col in dec.y_var.columns if col not in dec.y_var_reduced.columns]

In [ ]:
[col for col in dec.y_var_reduced.columns if col not in dec.y_var.columns]

## Check gpfa's binned spikes vs my own binned spikes

In [ ]:
spike_segs_df = fit_gpfa_utils.make_spike_segs_df(dec.spike_df, dec.single_vis_target_df)


### get binned spikes (seqs) from gpfa_utils.

In [ ]:
seg_index = 9
cluster_index = 12
seg = dec.spiketrain_corr_segs[seg_index]
cluster = dec.spike_segs_df.cluster.unique()[cluster_index]

spiketrain = dec.spiketrains[seg_index][cluster_index]
seqs = gpfa_util.get_seqs([spiketrain], dec.bin_width_w_unit)

### take out my own binned spikes

In [ ]:
p_sub = dec.pursuit_data_all[dec.pursuit_data_all['segment']==seg]
binned_spikes_sub = dec.binned_spikes_df[dec.binned_spikes_df['bin'].isin(p_sub['bin'])].copy()
binned_spikes_sub['bin'] = binned_spikes_sub.index
binned_spikes_sub2 = binned_spikes_sub.merge(p_sub[['bin', 'time']], on='bin', how='left')
binned_spikes_sub3 = binned_spikes_sub2[['bin', 'time', f'unit_{cluster}']].copy()

### compare

In [ ]:
trial_length = len(binned_spikes_sub3)
if dec.align_at_beginning:
    binned_spikes_sub3['gpfa'] = seqs[0][1][0][trial_length:]
else:
    binned_spikes_sub3['gpfa'] = seqs[0][1][0][-trial_length:] # when getting latent dimension for neural data, [-trial_length:] was also used
binned_spikes_sub3['same'] = binned_spikes_sub3[f'unit_{cluster}'] == binned_spikes_sub3['gpfa']
binned_spikes_sub3[binned_spikes_sub3['same']!=True]